In [25]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
import cv2

In [26]:
#Initialization of selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [27]:
#Path to images and folder with bounding boxes of all airplane locations in an image for all images
images_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images"
label_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Airplanes_Annotations"

In [28]:
#there are 700 images with name starting with airplane, and 33 with numbers
#to hold the image name and their bb of all airplane locations
img_bb = {}

In [29]:
for label in os.listdir(label_path):                        # It gives each element of label_path directory to here (label)
    if label.split("_")[0] == "airplane":                   # We are only considering these images
        try:
            file = pd.read_csv(label_path+"/"+label)        # Reading each csv file containing bb of each image
            # print("label: ", label)
            # print("len of file: ", len(file))
            labels_for_image = []                           # To store the bb in integer format from csv
            for row in file.iterrows():
                # for k in row:
                #     print(row[k][0].split(" "))
                bb_each = list(row[1])[0].split(" ")
                for j in range(len(bb_each)):
                    bb_each[j] = eval(bb_each[j])

                labels_for_image.append(bb_each)            # Labels_for_image is the list with all bb of current image
            img_bb[label.split(".")[0]] = labels_for_image  # We choose file name without jpg as key and labels_for_image as their value
        except Exception as e:
            print(e)
            print("Error in :", label)

In [30]:
# img_bb is a dictionary that now contains image name as key and their true bb of each airplane as a list of lists as the value of dictionary
print("Total images for training: ", len(img_bb))

Total images for training:  700


In [31]:
# To check how data is stored
c = 0
for i in img_bb:
    if c == 4:
        break
    print("Image name: ", i)
    print("Number of labels: ", len(img_bb[i]))
    print("Labels: ", img_bb[i])
    c += 1

Image name:  airplane_201
Number of labels:  26
Labels:  [[22, 78, 31, 90], [29, 69, 48, 81], [44, 65, 59, 76], [58, 57, 68, 70], [122, 16, 138, 26], [134, 9, 146, 21], [93, 35, 102, 46], [79, 41, 93, 56], [69, 48, 79, 59], [27, 110, 39, 121], [38, 102, 48, 111], [48, 97, 62, 105], [61, 89, 76, 97], [73, 82, 86, 91], [87, 75, 96, 84], [106, 69, 118, 79], [118, 53, 133, 65], [133, 47, 145, 58], [144, 39, 154, 48], [116, 238, 127, 253], [126, 232, 143, 248], [136, 225, 152, 241], [154, 216, 164, 226], [165, 209, 179, 219], [166, 220, 174, 230], [152, 229, 165, 237]]
Image name:  airplane_407
Number of labels:  3
Labels:  [[30, 90, 78, 131], [143, 6, 190, 46], [200, 180, 253, 236]]
Image name:  airplane_343
Number of labels:  2
Labels:  [[122, 153, 197, 247], [79, 5, 147, 98]]
Image name:  airplane_520
Number of labels:  4
Labels:  [[63, 116, 92, 137], [107, 91, 132, 115], [140, 128, 165, 156], [176, 230, 202, 254]]


In [32]:
ele1n = ""
elebb1 = []

In [33]:
# To calculate the area of intersection
def gt(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [35]:
'''
1   Reading each image from folder
2   we consider only images starting with name airplane
3   Performing selective search
4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]
        where (x, y) are top left corner coordinates of bb and w, h are width and height
5   Here we do not add width and height as the label data cause:
6   we downloaded has bb of the format [x1, y1, x2, y2]
        (x1, y1): top left corner coordinates
        (x2, y2): bottom right corner coordinates
7   Here iou is greater than 0.7, if we haven't added 30 of this class to train_images:
        add the resized image of this proposed bounding box by selective search to train_images



Steps involved in displaying an image in opencv:
    cv2.imshow("imout: ", imout)
    cv2.waitKey(0),
    cv2.destroyAllWindows()


'''

'\n1   Reading each image from folder\n2   we consider only images starting with name airplane\n3   Performing selective search\n4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]\n        where (x, y) are top left corner coordinates of bb and w, h are width and height\n5   Here we do not add width and height as the label data cause:\n6   we downloaded has bb of the format [x1, y1, x2, y2]\n        (x1, y1): top left corner coordinates\n        (x2, y2): bottom right corner coordinates\n7   Here iou is greater than 0.7, if we haven\'t added 30 of this class to train_images:\n        add the resized image of this proposed bounding box by selective search to train_images\n\n\n\nSteps involved in displaying an image in opencv:\n    cv2.imshow("imout: ", imout)\n    cv2.waitKey(0),\n    cv2.destroyAllWindows()\n\n\n'

In [61]:
#we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals

x_new_images, y_new_labels = [], []
no_images = 0
no_images_to_be_used_for_extraction = 30
total_images_extracted = 0
for img in os.listdir(images_path):             # 1
    if no_images == no_images_to_be_used_for_extraction:   # We are only 10 images, 30 resized images each for foreground and background are taken from each of these 10 images
                                                # Total of 300 positive images and 300 negative samples are given for VGGNet to train
        break
    try:
        if img.split("_")[0] == "airplane":                              # 2

            image = cv2.imread(os.path.join(images_path, img))

            ss.setBaseImage(image)                                       # 3
            ss.switchToSelectiveSearchFast(base_k=10)
            ssresults = ss.process()
            imout = image.copy()

            # cv2.imshow("imout: ", imout)
            # cv2.waitKey(0),
            # cv2.destroyAllWindows()


            # ele1n = os.path.join(images_path, img)

            # In openCV, consider a square window facing us, 
            # its left side will be y axis and top side will be x axis
            # top left corner is considered as (0, 0)

            iou_07 = 0
            iou_03 = 0
            for i in ssresults:
                x1, y1, w, h = i                                          # 4
                bb2 = {"x1":x1, "y1":y1, "x2":x1+w, "y2":y1+h}            
                img = img.split(".")[0]
                for j in img_bb[img]:                                     # For each true bb in img_bb, we calculate iou for the bb region proposed by ss
                    x, y, xn, yn = j                                      # 5
                    bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}              # 6
                    iou = gt(bb1, bb2)                                    # Calculating iou
                    if iou > 0.7 and iou_07 < 30:                         # 7
                        # print("iou07: bb: x1, y1, w, h, x1+w, y1+h:", x1, y1, w, h, x1+w, y1+h)
                        timage = imout[y1:y1+h,x1:x1+w]
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        x_new_images.append(resized)
                        y_new_labels.append(1)
                        iou_07 += 1
                        # cv2.imshow("resized: ", resized)
                        # cv2.waitKey(0)
                        # cv2.destroyAllWindows()
                        # cv2.rectangle(image, (bb1["x1"], bb1["y1"]), (bb1["x2"], bb1["y2"]), (255, 0, 0), 2)
                        # cv2.rectangle(image, (bb2["x1"], bb2["y1"]), (bb2["x2"], bb2["y2"]), (0, 255, 0), 1)
                    elif iou < 0.3 and iou_03 < 30:
                        timage = imout[y1:y1+h, x1:x1+h]
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        x_new_images.append(resized)
                        y_new_labels.append(0)
                        iou_03 += 1
            # cv2.imshow(img, image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # print("for image: ", img, " there are _ iou07 and _ iou03: ", iou_07, iou_03)
            total_images_extracted += iou_03 + iou_07
        no_images += 1
    except Exception as e:
        print(e)
        print("Error in image: ", img)
        continue

In [37]:
# These lines can display the image you want

# i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/airplane_343.jpg")
# cv2.imshow("image", i)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [62]:
# Now we have in total 1246 images of positive and negative samples mix to fine tune the next VGGNet model

print("total images extracted and resized: ", total_images_extracted)
print("len of x_new_images: ", len(x_new_images))
print("len of y_new_labels: ", len(y_new_labels))

total images extracted and resized:  1246
len of x_new_images:  1246
len of y_new_labels:  1246


In [55]:
# Displaying images of x_new_images resized images:

# for index, i in enumerate(x_new_images):
#     cv2.imshow("image: " + str(y_new_labels[index]), i)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()